# Gradient checks

It is best practice to do gradient checks before and after gradient-based optimization.
Here we show, how to use the gradient check methods that are implemented in pyPESTO, using the finite differences (FD) method as a comparison.
There is a trade-off between the quality of the approximation and numerical noise, so it is recommended to try different FD step sizes.  
Most importantly, test your gradients using the settings you will use later on.

In [1]:
import numpy as np

import benchmark_models_petab as models
import pypesto.optimize as optimize
import pypesto.petab

np.random.seed(2)

import seaborn as sns

#### Set up an example problem

Create the pypesto problem and a random vector of parameter values.

In [2]:
%%capture

model_name = "Boehm_JProteomeRes2014"
petab_problem = models.get_problem(model_name)

importer = pypesto.petab.PetabImporter(petab_problem)
pypesto_problem = importer.create_problem(verbose=False)

### Gradient check before optimization
Here we use the startpoint sampling method to generate random parameter vectors.

In [3]:
startpoints = pypesto_problem.get_startpoints(n_starts=2)

Perform a gradient check at the location of one of the parameter vectors. `check_grad` compares the gradients obtained by the finite differences (FD) method and the objective gradient. You can modify the finite differences step size via the argument `eps`.

In [4]:
pypesto_problem.objective.check_grad(
    x = startpoints[0], 
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
Epo_degradation_BaF3,2.899805e+10,2.898349e+10,2.899516e+10,2.898933e+10,1.166055e+07,8.729236e+06,3.011190e-04
k_exp_hetero,-1.822477e+03,8.247990e+07,-1.185836e+08,-1.805185e+07,2.010635e+08,1.805003e+07,9.998990e-01
k_exp_homo,1.940159e+06,-7.634094e+06,2.620560e+07,9.285754e+06,3.383970e+07,7.345596e+06,7.910607e-01
k_imp_hetero,1.324222e+09,9.636109e+08,1.401833e+09,1.182722e+09,4.382216e+08,1.415004e+08,1.196396e-01
k_imp_homo,2.759777e+09,2.689595e+09,2.810697e+09,2.750146e+09,1.211021e+08,9.630702e+06,3.501887e-03
k_phos,-3.183894e+10,-3.189761e+10,-3.181251e+10,-3.185506e+10,8.509938e+07,1.611509e+07,5.058878e-04
sd_pSTAT5A_rel,-4.106435e+12,-4.106388e+12,-4.106482e+12,-4.106435e+12,9.372550e+07,6.663599e+02,1.622721e-10
sd_pSTAT5B_rel,-2.467665e+11,-2.468085e+11,-2.467245e+11,-2.467665e+11,8.401874e+07,3.362799e+01,1.362745e-10
sd_rSTAT5A_rel,3.684015e+01,-4.769135e+07,4.769149e+07,7.324219e+01,9.538284e+07,3.640203e+01,4.970090e-01


Explanation of the gradient check result columns:
- `grad`: Objective gradient
- `fd_f`: FD forward difference
- `fd_b`: FD backward difference
- `fd_c`: Approximation of FD central difference (reusing the information from `fd_f` and `fd_b`)
- `fd_err`: Deviation between forward and backward differences `fd_f`, `fd_b`
- `abs_err`: Absolute error between `grad` and the central FD gradient `fd_c`
- `rel_err` Relative error between `grad` and the central FD gradient `fd_c`

If there are fixed parameters in your vector you might invoke an error due to the dimension mismatch. Use the helper method `Problem.get_reduced_vector` to get the reduced vector with only free (estimated) parameters:

In [5]:
pypesto_problem.objective.check_grad(
    x = pypesto_problem.get_reduced_vector(startpoints[0]),
    eps = 1e-5,  # default
    verbosity = 0,
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err
Epo_degradation_BaF3,2.899805e+10,2.898349e+10,2.899516e+10,2.898933e+10,1.166055e+07,8.729236e+06,3.011190e-04
k_exp_hetero,-1.822477e+03,8.247990e+07,-1.185836e+08,-1.805185e+07,2.010635e+08,1.805003e+07,9.998990e-01
k_exp_homo,1.940159e+06,-7.634094e+06,2.620560e+07,9.285754e+06,3.383970e+07,7.345596e+06,7.910607e-01
k_imp_hetero,1.324222e+09,9.636109e+08,1.401833e+09,1.182722e+09,4.382216e+08,1.415004e+08,1.196396e-01
k_imp_homo,2.759777e+09,2.689595e+09,2.810697e+09,2.750146e+09,1.211021e+08,9.630702e+06,3.501887e-03
k_phos,-3.183894e+10,-3.189761e+10,-3.181251e+10,-3.185506e+10,8.509938e+07,1.611509e+07,5.058878e-04
sd_pSTAT5A_rel,-4.106435e+12,-4.106388e+12,-4.106482e+12,-4.106435e+12,9.372550e+07,6.663599e+02,1.622721e-10
sd_pSTAT5B_rel,-2.467665e+11,-2.468085e+11,-2.467245e+11,-2.467665e+11,8.401874e+07,3.362799e+01,1.362745e-10
sd_rSTAT5A_rel,3.684015e+01,-4.769135e+07,4.769149e+07,7.324219e+01,9.538284e+07,3.640203e+01,4.970090e-01


Next, we do optimization and perform a gradient check at a local optimum.  
The method `check_grad_multi_eps` calls the `check_grad` method above multiple times with different settings for the FD step size and reports the setting that results in the smallest error. 
You can supply a list of FD step sizes to be tested via the `multi_eps` argument (or use the default ones), and use the `label` argument to switch between the FD, or absolute or relative error.

In [6]:
%%capture

result = optimize.minimize(
    problem=pypesto_problem, 
    optimizer=optimize.ScipyOptimizer(), 
    n_starts=4,
)

In [7]:
gc = pypesto_problem.objective.check_grad_multi_eps(
    x=pypesto_problem.get_reduced_vector(result.optimize_result[0].x),
    verbosity=0,
    label='rel_err',  # default
)
gc

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
Epo_degradation_BaF3,-0.001558,1.100138,-1.104286,-0.002074,2.204424,0.000515,0.479959,1.000000e-03
k_exp_hetero,0.055354,0.062354,0.049370,0.055862,0.012984,0.000508,0.003259,1.000000e-01
k_exp_homo,0.000609,0.033696,-0.032193,0.000751,0.065888,0.000142,0.081331,1.000000e-03
k_imp_hetero,-0.001165,197.943505,-181.738627,8.102439,379.682131,8.103604,0.987950,1.000000e-01
k_imp_homo,-0.000044,-0.000033,-0.000074,-0.000053,0.000041,0.000009,0.000093,1.000000e-01
k_phos,-0.001786,0.477147,-0.480739,-0.001796,0.957886,0.000010,0.012976,1.000000e-03
sd_pSTAT5A_rel,0.000435,0.187172,-0.186303,0.000434,0.373475,0.000001,0.002282,1.000000e-05
sd_pSTAT5B_rel,0.000220,0.186953,-0.186522,0.000216,0.373475,0.000004,0.019104,1.000000e-05
sd_rSTAT5A_rel,-0.000259,18.588643,-18.589168,-0.000262,37.177811,0.000003,0.011911,1.000000e-07


Use the pandas style methods to visualise the results of the gradient check, e.g.:

In [8]:
def highlight_value_above_threshold(x, threshold=10):
    return ['color: darkorange' if xi > threshold else None for xi in x]

gc.style.apply(
    highlight_value_above_threshold, subset=["fd_err"],
).background_gradient(
    cmap=sns.light_palette("purple", as_cmap=True), subset=["abs_err"],
).background_gradient(
    cmap=sns.light_palette("red", as_cmap=True), subset=["rel_err"],
)

,grad,fd_f,fd_b,fd_c,fd_err,abs_err,rel_err,eps
Epo_degradation_BaF3,-0.001558,1.100138,-1.104286,-0.002074,2.204424,0.000515,0.479959,0.001000
k_exp_hetero,0.055354,0.062354,0.049370,0.055862,0.012984,0.000508,0.003259,0.100000
k_exp_homo,0.000609,0.033696,-0.032193,0.000751,0.065888,0.000142,0.081331,0.001000
k_imp_hetero,-0.001165,197.943505,-181.738627,8.102439,379.682131,8.103604,0.987950,0.100000
k_imp_homo,-0.000044,-0.000033,-0.000074,-0.000053,0.000041,0.000009,0.000093,0.100000
k_phos,-0.001786,0.477147,-0.480739,-0.001796,0.957886,0.000010,0.012976,0.001000
sd_pSTAT5A_rel,0.000435,0.187172,-0.186303,0.000434,0.373475,0.000001,0.002282,0.000010
sd_pSTAT5B_rel,0.000220,0.186953,-0.186522,0.000216,0.373475,0.000004,0.019104,0.000010
sd_rSTAT5A_rel,-0.000259,18.588643,-18.589168,-0.000262,37.177811,0.000003,0.011911,0.000000


### How to "fix" my gradients?

- Find suitable simulation tolerances.
- Consider switching from adjoint to forward sensitivities, which tend to be more robust.
- Check the simulation logs for Warnings and Errors.
- Ensure that the model is correct.
- Be aware that, depending on the tolerances, the gradient might differ when it is calculated with sensitivities, because `Objective(x, sensi_orders=(0,))` might provide a different value than `Objective(x, sensi_orders=(0,1))`